In [1]:
import numpy as np
import pandas as pd

# 1.移位（Shifting）:

In [2]:
ts = pd.Series(np.arange(1,6) , index=pd.date_range("2022-12-12" , periods=5))
ts

2022-12-12    1
2022-12-13    2
2022-12-14    3
2022-12-15    4
2022-12-16    5
Freq: D, dtype: int32

In [3]:
ts.shift(2)

2022-12-12    NaN
2022-12-13    NaN
2022-12-14    1.0
2022-12-15    2.0
2022-12-16    3.0
Freq: D, dtype: float64

In [4]:
ts.shift(-2)

2022-12-12    3.0
2022-12-13    4.0
2022-12-14    5.0
2022-12-15    NaN
2022-12-16    NaN
Freq: D, dtype: float64

### shift 的一个常见用途是计算时间序列或DataFrame 多列时间序列的百分比变化：

In [5]:
ts / ts.shift(1) - 1

2022-12-12         NaN
2022-12-13    1.000000
2022-12-14    0.500000
2022-12-15    0.333333
2022-12-16    0.250000
Freq: D, dtype: float64

## 1.1 传递频率以推移时间戳

In [6]:
ts

2022-12-12    1
2022-12-13    2
2022-12-14    3
2022-12-15    4
2022-12-16    5
Freq: D, dtype: int32

### 如果频率已知，则可以将其传递给 shift 以推移时间戳，而不仅仅是数据：

In [7]:
ts.shift(2 , freq="D")

2022-12-14    1
2022-12-15    2
2022-12-16    3
2022-12-17    4
2022-12-18    5
Freq: D, dtype: int32

### 也可以传递其他频率，让你可以灵活地移位数据：

In [8]:
ts.shift(2 , freq="1h30min")

2022-12-12 03:00:00    1
2022-12-13 03:00:00    2
2022-12-14 03:00:00    3
2022-12-15 03:00:00    4
2022-12-16 03:00:00    5
Freq: D, dtype: int32

In [9]:
# 这里的 T 代表分钟。
ts.shift(2,freq="90T")

2022-12-12 03:00:00    1
2022-12-13 03:00:00    2
2022-12-14 03:00:00    3
2022-12-15 03:00:00    4
2022-12-16 03:00:00    5
Freq: D, dtype: int32

# 2.使用偏置移位日期：

### pandas 日期偏量也可以使用 datetime 或 Timestamp 对象：

In [10]:
from pandas.tseries.offsets import Day , MonthEnd
from datetime import datetime

now = datetime(2022,12,12)
now + 3*Day()

Timestamp('2022-12-15 00:00:00')

### 如果添加像 MonthEnd 这样的锚定偏置量，第一个增量将根据频率规则将日期“前滚”到下一个日期：

In [11]:
now + MonthEnd()

Timestamp('2022-12-31 00:00:00')

In [12]:
now + MonthEnd(2)

Timestamp('2023-01-31 00:00:00')

### 锚定的偏置量可以使用 rollforward 和 rollback 方法将日期显式地向前或向后“滚动”：

In [13]:
offset = MonthEnd()

In [14]:
offset.rollforward(now)

Timestamp('2022-12-31 00:00:00')

In [15]:
offset.rollback(now)

Timestamp('2022-11-30 00:00:00')

# 3.将移位方法与 groupby 一起使用是日期偏置的一种创造性用法：

In [16]:
ts = pd.Series(np.arange(20) , index=pd.date_range("2022-12-12" , periods=20 , freq="4D"))
ts

2022-12-12     0
2022-12-16     1
2022-12-20     2
2022-12-24     3
2022-12-28     4
2023-01-01     5
2023-01-05     6
2023-01-09     7
2023-01-13     8
2023-01-17     9
2023-01-21    10
2023-01-25    11
2023-01-29    12
2023-02-02    13
2023-02-06    14
2023-02-10    15
2023-02-14    16
2023-02-18    17
2023-02-22    18
2023-02-26    19
Freq: 4D, dtype: int32

In [17]:
grouped = ts.groupby(MonthEnd().rollforward)

In [18]:
for name,value in grouped:
    print(name)
    print(value)

2022-12-31 00:00:00
2022-12-12    0
2022-12-16    1
2022-12-20    2
2022-12-24    3
2022-12-28    4
Freq: 4D, dtype: int32
2023-01-31 00:00:00
2023-01-01     5
2023-01-05     6
2023-01-09     7
2023-01-13     8
2023-01-17     9
2023-01-21    10
2023-01-25    11
2023-01-29    12
Freq: 4D, dtype: int32
2023-02-28 00:00:00
2023-02-02    13
2023-02-06    14
2023-02-10    15
2023-02-14    16
2023-02-18    17
2023-02-22    18
2023-02-26    19
Freq: 4D, dtype: int32


In [19]:
grouped.mean()

2022-12-31     2.0
2023-01-31     8.5
2023-02-28    16.0
dtype: float64

### 更简单快捷的方法是使用后续会介绍的resample 方法（重采样）：

In [20]:
ts.resample("M").mean()

2022-12-31     2.0
2023-01-31     8.5
2023-02-28    16.0
Freq: M, dtype: float64